## Black-Scholes (BS) Partial Differential Equation (PDE) America/European options

#### BS PDE governs the dynamics of option pricing:

$$
\frac{\partial V}{\partial t} + \frac{1}{2} \sigma^2 S^2 \frac{\partial^2 V}{\partial S^2} + rS \frac{\partial V}{\partial S} - rV = 0
$$

#### with dividend yield:

$$
\frac{\partial V}{\partial t} + \frac{1}{2} \sigma^2 S^2 \frac{\partial^2 V}{\partial S^2} + (r - q)S \frac{\partial V}{\partial S} - rV = 0
$$


To derive the Black-Scholes Partial Differential Equation (PDE), we start with the stochastic differential equation (SDE) that describes the dynamics of the underlying asset in the Black-Scholes model:

$$ dS_t = \mu S_t dt + \sigma S_t dW_t $$

Where: 


$S_t$ - asset price at time t \
$\mu$ - drift rate \
$\sigma$ - volatility \
$( dW_t )$ - Wiener process or Brownian motion 


We aim to model a derivative's value c($S_t, t$).
Where c is contingent on the asset price and time.
Let's denote the payoff at maturity for the derivative as $c(S_t, T)$. 
For example, in the case of a European call option, the payoff at maturity would be $max(S_T - K,0)$, where K is the strike price. 

Applying Ito's Lemma to $c(S_t ,t)$ using $( dS_t )$ from the above SDE yields:

$$ dc(S_t, t) = \frac{\partial c}{\partial s} dS_t + \frac{1}{2} \frac{\partial^2 c}{\partial s^2} (dS_t)^2 + \frac{\partial c}{\partial t} dt $$
Ignoring higher-order terms and substituting the SDE for \(dS_t\), we get:

$$
dc(S_t, t) = \left(\frac{\partial c}{\partial s} + \frac{1}{2} \sigma^2 S_t^2 \frac{\partial^2 c}{\partial s^2}\right) dt + \frac{\partial c}{\partial s} \sigma S_t dW_t
$$


This equation describes how the option value $c(S_t, t)$ evolves over time. Now, let's consider a portfolio that includes both the derivative and the underlying asset $(S_t)$ to hedge the risk in $c(S_t, t)$. We aim to eliminate the \(dW_t\) term by forming a position in $(S_t)$.

The exposed term to randomness is $(\frac{\partial c}{\partial s} \sigma S_t dW_t)$. Owning one unit of the derivative, $c(S_t, t)$, and $(-\frac{\partial c}{\partial s})$ units of $(S_t)$ makes the portfolio immune to $(dW_t)$. This portfolio, referred to as a delta-hedged portfolio, can be written as:

$$
\Pi(S_t, t) = c(S_t, t) - \frac{\partial c}{\partial s} S_t
$$

The evolution of $\Pi(S_t, t)$ over time $(dt)$ is described by:

$$
d \Pi(S_t, t) = \left(\frac{\partial c}{\partial t} + \mu S_t \frac{\partial c}{\partial s} + \frac{1}{2} \sigma^2 S_t^2 \frac{\partial^2 c}{\partial s^2}\right) dt
$$


This portfolio is risk-free because of the hedge, implying it should earn the risk-free rate of return. \
Equating this with the change in portfolio value $(d \Pi)$, we derive the Black-Scholes PDE:
$$
\frac{\partial V}{\partial t} + \frac{1}{2} \sigma^2 S^2 \frac{\partial^2 V}{\partial S^2} + rS \frac{\partial V}{\partial S} - rV = 0
$$
This equation describes the evolution of option value over time in the Black-Scholes model, considering factors such as time, asset price, volatility, and risk-free rate. It's a fundamental equation in option pricing, providing insights into option dynamics and relationships between various parameters. Though most finance PDEs require numerical methods for solutions, the Black-Scholes PDE is an exception, allowing analytical solutions through techniques like change of variables.


In [1]:
import numpy as np
import scipy.stats as ss
from scipy.sparse.linalg import spsolve
from scipy import sparse
from scipy.sparse.linalg import splu
from time import time

## BS closed form solution for comparisson

In [2]:
def closed_formula(S0, K, T, r, q, sig, payoff):
    """
    Black Scholes closed formula for option pricing.
   
    Parameters:
        S0: float, current stock price
        K: float, strike price
        T: float, time to maturity
        r: float, risk-free rate
        q: float, dividend yield
        sig: float, volatility of the underlying asset
        payoff: str, type of option ("call" or "put")
       
    Returns:
        float, option price
    """
    d1 = (np.log(S0 / K) + (r - q + sig**2 / 2) * T) / (sig * np.sqrt(T))
    d2 = d1 - sig * np.sqrt(T)

    if payoff == "call":
        return S0 * np.exp(-q * T) * ss.norm.cdf(d1) - K * np.exp(-r * T) * ss.norm.cdf(d2)
    elif payoff == "put":
        return K * np.exp(-r * T) * ss.norm.cdf(-d2) - S0 * np.exp(-q * T) * ss.norm.cdf(-d1)
    else:
        raise ValueError("Invalid type. Set 'call' or 'put'")

## BS PDE solution

In [3]:
def PDE_price(S0, K, T, r, q, sig, steps, payoff="call", exercise="European", solver="splu", Time = False):
    """
    Computes option price using finite-difference Black-Scholes PDE.
   
    Parameters:
        S0: float, current stock price
        K: float, strike price
        T: float, time to maturity
        r: float, risk-free rate
        q: float, dividend yield
        sig: float, volatility of the underlying asset
        steps: tuple, number of space and time steps (Nspace, Ntime)
        payoff: str, type of option ("call" or "put")
        exercise: str, type of exercise ("European" or "American")
        solver: str, type of solver ("splu" or "spsolve")
       
    Returns:
        float, option price
    """
    t_init = time()

    Nspace = steps[0]
    Ntime = steps[1]

    S_max = 6 * float(K)
    S_min = float(K) / 6
    x_max = np.log(S_max)
    x_min = np.log(S_min)
    x0 = np.log(S0)  # current log-price

    x, dx = np.linspace(x_min, x_max, Nspace, retstep=True)
    t, dt = np.linspace(0, T, Ntime, retstep=True)

    S_vec = np.exp(x)  # vector of S
    Payoff = np.maximum(S_vec - K, 0) if payoff == "call" else np.maximum(K - S_vec, 0)

    V = np.zeros((Nspace, Ntime))
    if payoff == "call":
        V[:, -1] = Payoff
        V[-1, :] = np.exp(x_max) - K * np.exp(-r * t[::-1])
        V[0, :] = 0
    else:
        V[:, -1] = Payoff
        V[-1, :] = 0
        V[0, :] = Payoff[0] * np.exp(-r * t[::-1])  

    sig2 = sig**2
    dxx = dx**2
    a = (dt / 2) * ((r - q - 0.5 * sig2) / dx - sig2 / dxx)
    b = 1 + dt * (sig2 / dxx + r)
    c = -(dt / 2) * ((r - q - 0.5 * sig2) / dx + sig2 / dxx)

    D = sparse.diags([a, b, c], [-1, 0, 1], shape=(Nspace - 2, Nspace - 2)).tocsc()

    offset = np.zeros(Nspace - 2)

    if solver == "spsolve":
        if exercise == "European":
            for i in range(Ntime - 2, -1, -1):
                offset[0] = a * V[0, i]
                offset[-1] = c * V[-1, i]
                V[1:-1, i] = spsolve(D, (V[1:-1, i + 1] - offset))
        elif exercise == "American":
            for i in range(Ntime - 2, -1, -1):
                offset[0] = a * V[0, i]
                offset[-1] = c * V[-1, i]
                V[1:-1, i] = np.maximum(spsolve(D, (V[1:-1, i + 1] - offset)), Payoff[1:-1])

    elif solver == "splu":
        DD = splu(D)
        if exercise == "European":
            for i in range(Ntime - 2, -1, -1):
                offset[0] = a * V[0, i]
                offset[-1] = c * V[-1, i]
                V[1:-1, i] = DD.solve(V[1:-1, i + 1] - offset)
        elif exercise == "American":
            for i in range(Ntime - 2, -1, -1):
                offset[0] = a * V[0, i]
                offset[-1] = c * V[-1, i]
                V[1:-1, i] = np.maximum(DD.solve(V[1:-1, i + 1] - offset), Payoff[1:-1])
    else:
        raise ValueError("Solver is splu or spsolve")

    price = np.interp(x0, x, V[:, 0])

    if Time is True:
        elapsed = time() - t_init
        return price, elapsed
    else:
        return price

In [8]:
# Define option parameters
S0 = 100  # current stock price
K = 100   # strike price
T = 1    # time to maturity (in years)
r = 0.1 # risk-free rate
q = 0.03   # dividend yield
sig = 0.35 # volatility of the underlying asset
steps = (2000, 20000)  # number of space and time steps, **I find increased number of time steps to improve the result**
payoff = "call"  # type of option ("call" or "put")
exercise = "European"  # type of exercise ("European" or "American")

# Compute option price using closed-form formula
price_closed_form = closed_formula(S0, K, T, r, q, sig, payoff)
print("Option price using closed-form formula:", price_closed_form)

# Compute option price using finite-difference Black-Scholes PDE
price_PDE = PDE_price(S0, K, T, r, q, sig, steps, payoff, exercise, "splu", True)
print("Option price using finite-difference Black-Scholes PDE:", price_PDE[0], "Time:", price_PDE[1], "splu solver much faster run a slow PC")
price_PDE_1 = PDE_price(S0, K, T, r, q, sig, steps, payoff, exercise, "spsolve", True)
print("Option price using finite-difference Black-Scholes PDE:", price_PDE_1[0], "Time:", price_PDE_1[1])
print("Difference:", "{:.6f}".format(price_PDE[0] - price_closed_form))

Option price using closed-form formula: 17.850548877543837
Option price using finite-difference Black-Scholes PDE: 17.850516456555013 Time: 3.4829864501953125 splu solver much faster run a slow PC
Option price using finite-difference Black-Scholes PDE: 17.850516456555013 Time: 67.93569421768188
Difference: -0.000032


In [12]:
print(PDE_price(S0, K, T, r, q, sig, steps, 'put', 'European', "splu", True))
print(PDE_price(S0, K, T, r, q, sig, steps, 'put', "American", "splu", True))

(9.329254191825532, 4.104466676712036)
(10.3826014965317, 3.6549758911132812)


#### Literature used

###### https://github.com/cantaro86/Financial-Models-Numerical-Methods/blob/master/src/FMNM/BS_pricer.py   -> with modification in the code, and added the dividends yield parameter

###### C. Kelliher (2022) - Quantitative Finance With Python A Practical Guide to Investment Management, Trading, and Financial Engineering
###### Wilmott Paul (1994). Option pricing: Mathematical models and computation. Oxford Financial Press.